# Use ChromaDocumentStore with Haystack

https://docs.haystack.deepset.ai/docs/chromadocumentstore

WORKING (chromadb2 env)

In [1]:
# Install the Chroma integration, Haystack will come as a dependency
# !pip install -U chroma-haystack "huggingface_hub>=0.22.0"

#!pip install chroma-haystack
#!pip install torch torchaudio torchvision

## Indexing Pipeline: preprocess, split and index documents
In this section, we will index documents into a Chroma DB collection by building a Haystack indexing pipeline. Here, we are indexing documents from the [VIM User Manuel](https://vimhelp.org/) into the Haystack `ChromaDocumentStore`.

 We have the `.txt` files for these pages in the examples folder for the `ChromaDocumentStore`, so we are using the [`TextFileToDocument`](https://docs.haystack.deepset.ai/v2.0/docs/textfiletodocument) and [`DocumentWriter`](https://docs.haystack.deepset.ai/v2.0/docs/documentwriter) components to build this indexing pipeline.

In [2]:
import os
from pathlib import Path
import torch
torch.cuda.is_available()

True

In [3]:
from haystack import Pipeline
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack import Document

In [4]:
from rich import print  # https://rich.readthedocs.io/en/stable/markup.html#console-markup
from rich.console import Console
from rich.table import Table

console = Console()
print(f"Torchy?? [bold red]{torch.cuda.is_available()}[/]")
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"


Torchy?? True

In [5]:
# file_paths = ["data" / Path(name) for name in os.listdir("data")]
from pathlib import Path
data_dir = "../data/text"
text_files = list(Path(data_dir).glob("**/*.txt"))
print(text_files)

[PosixPath('../data/test.txt'), PosixPath('../data/.ipynb_checkpoints/test-checkpoint.txt')]

In [ ]:
document_store = ChromaDocumentStore()

# remove repeated substrings to get rid of headers/footers
cleaner = DocumentCleaner(remove_repeated_substrings=True)
# Since jina-v2 can handle 8192 tokens, 500 words seems like a safe chunk size
splitter = DocumentSplitter(split_by="word", split_length=100)
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
document_embedder = SentenceTransformersDocumentEmbedder(model=embedding_model)
document_writer = DocumentWriter(document_store)

In [ ]:
indexing = Pipeline()
indexing.add_component("converter", TextFileToDocument())
indexing.add_component(instance=cleaner, name="document_cleaner")
indexing.add_component(instance=splitter, name="document_splitter")
indexing.add_component(instance=document_embedder, name="document_embedder")
indexing.add_component(instance=document_writer, name="document_writer")

In [ ]:
indexing.connect("converter", "document_cleaner")
indexing.connect("document_cleaner", "document_splitter")
indexing.connect("document_splitter", "document_embedder")
indexing.connect("document_embedder", "document_writer")

In [ ]:
indexing.run({"converter": {"sources": text_files}})

In [ ]:
# Create pipeline components
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

retriever = ChromaQueryTextRetriever(document_store=document_store, top_k=3)
text_embedder = SentenceTransformersTextEmbedder(model=embedding_model)
text_embedder.warm_up()

In [ ]:
query_pipeline = Pipeline()
# query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)

In [ ]:
# test components...
foo = retriever.run(query="test me")

In [ ]:
len(foo["documents"])

In [ ]:
q = "How many languages are there?"
k = 5
query_embedding = text_embedder.run(q)

In [ ]:
# result = query_pipeline.run({"text_embedder": {"text": query}})
result = query_pipeline.run({"retriever": {"query": q, "top_k": k}})
# result = query_pipeline.run(query = query)

In [ ]:
hl_list = [
        r"(?i)(?P<learn>ai|learn[a-z]*|lesson|ml|train[a-z]*|recogni[a-z]*|analy[a-z]*)",
        r"(?i)(?P<machine>(machine|computer|bot|agent|system|recommenda|speech|voice)[a-z]*)",
        r"(?i)(?P<model>embed[a-z]*|large|language|model[a-z]*|ml|llm|generat[a-z]*)",
        r"(?i)(?P<orgs>firm[a-z]*|org[a-z]*|compan[a-z]*|instit[a-z]*)",
        r"(?i)(?P<actions>(chat|search|build|generate|follow|create|use)[a-z]*)",
    ]

class MyHighlighter(RegexHighlighter):
    """Apply style to anything that looks like an email."""
    base_style = "example."
    highlights = hl_list

my_ml = MyHighlighter()

theme = Theme(
    {"example.learn": "bold magenta",
     "example.machine": "bold green",
     "example.model": "bold blue",
     "example.orgs": "bold yellow",
     "example.actions": "bold red",
     })
console.theme(theme)

In [ ]:
def display_results(table_title, results, show_lines=True, ):
    table = Table(title=table_title, show_lines=show_lines)
    # table.add_column("id", style="blue", )  # no_wrap=True
    table.add_column("Score", style="blue", )
    table.add_column("text")  # no_wrap=True
    for d in results["retriever"]["documents"]:
        # print(f"Result: ", d.meta, d.score)
        table.add_row("{:.2f}".format(d.score), d.content.strip(), )
    console.print(table)
    print('\n\n')

In [ ]:
title = f"Embedding model: [bold blue]{embedding_model}[/] \nQuery: [bold blue]{q}[/]"
display_results(title, result)

## Query Pipeline: build retrieval-augmented generation (RAG) pipelines

Once we have documents in the `ChromaDocumentStore`, we can use the accompanying Chroma retrievers to build a query pipeline. The query pipeline below is a simple retrieval-augmented generation (RAG) pipeline that uses Chroma's [query API](https://docs.trychroma.com/usage-guide#querying-a-collection).

You can change the idnexing pipeline and query pipelines here for embedding search by using one of the [`Haystack Embedders`](https://docs.haystack.deepset.ai/v2.0/docs/embedders) accompanied by the  `ChromaEmbeddingRetriever`.


In this example we are using:
- The `HuggingFaceTGIGenerator` with the Mistral-7B-Instruct-v0.1. (You will need a Hugging Face token to use this model). You can repleace this with any of the other [`Generators`](https://docs.haystack.deepset.ai/v2.0/docs/generators)
- The `PromptBuilder` which holds the prompt template. You can adjust this to a prompt of your choice
- The `ChromaQueryRetriver` which expects a list of queries and retieves the `top_k` most relevant documents from your Chroma collection.

In [ ]:
import os
from getpass import getpass
hfat = "HF_API_TOKEN"
if hfat in os.environ:
    print("---------- Found hugging face token in environ, no need to prompt")
    hf_token = os.environ["HF_API_TOKEN"]
else:
    print("++++++++++ Found hugging face token NOT in environ, need to prompt...")
    hf_token = getpass("Enter Hugging Face API key:")
    os.environ["HF_API_TOKEN"] = hf_token

In [ ]:
from haystack.components.builders import PromptBuilder

prompt = """
Answer the query based on the provided context.
If the context does not contain the answer, say 'Answer not found'.
Context:
{% for doc in documents %}
  {{ doc.content }}
{% endfor %}
query: {{query}}
Answer:
"""
prompt_builder = PromptBuilder(template=prompt)

In [ ]:
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack.components.generators import HuggingFaceTGIGenerator
from haystack.utils import Secret

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
stoken = Secret.from_token(hf_token)
# client = HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-v0.1", token=Secret.from_token(hf_token)
client = HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-Instruct-v0.2", token=stoken)
# HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-v0.1", token=Secret.from_token(

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
client.warm_up()
response = client.run("What's Natural Language Processing?")

print(response)

In [ ]:
llm = HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-Instruct-v0.1")
llm.warm_up()
retriever = ChromaQueryTextRetriever(document_store)

querying = Pipeline()
querying.add_component("retriever", retriever)
querying.add_component("prompt_builder", prompt_builder)
querying.add_component("llm", llm)

querying.connect("retriever.documents", "prompt_builder.documents")
querying.connect("prompt_builder", "llm")

In [ ]:
query = "What is the Revenue Capacity for jacksonville beach?"
query = "How is annual enrollment assessed in Jax Beach schools?"
# NOTE / TODO: typo our outdated synax from example?? --> "retriever": {"queries": [query]... is wrong/broken
results = querying.run({"retriever": {"query": query, "top_k": 3},
                        "prompt_builder": {"query": query},
                        "llm":{"generation_kwargs": {"max_new_tokens": 350}}})

In [ ]:
print(results["llm"]["replies"][0])